In [1]:
import os 
import re
import cantera as ct
import rmgpy.chemkin
import warnings


In [2]:
list_of_blends = [file for file in os.listdir('/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts') if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
list_of_blends                                                                                                                        

['C2H5F_CH2F2',
 'C2H5F_CH2FCH2F',
 'C2H5F_CH2FCHF2',
 'C2H5F_CH3CHF2',
 'C2H5F_CH3F',
 'CH2F2_CH3CF3',
 'CH2F2_CH3F',
 'CH2FCH2F_CH2F2',
 'CH2FCH2F_CH2FCHF2',
 'CH2FCH2F_CH3CHF2',
 'CH2FCH2F_CH3F',
 'CH2FCHF2_CH2F2',
 'CH2FCHF2_CH3F',
 'CH3CF3_C2H5F',
 'CH3CF3_CH2FCH2F',
 'CH3CF3_CH2FCHF2',
 'CH3CF3_CH3CHF2',
 'CH3CHF2_CH2F2',
 'CH3CHF2_CH2FCHF2',
 'CH3F_CH3CF3',
 'CH3F_CH3CHF2']

In [3]:
#dictionary with composition of each species 

compositions = {'C2H5F':{'C': 2,'H': 5,'F': 1},
                 'C2H6':{'C': 2,'H': 6},
                 'CH2F2': {'C':1,'H':2,'F':2},
                 'CH2FCH2F':{'C':2,'H':4,'F':2},
                 'CH2FCHF2': {'C':2,'H':3,'F':3},
                 'CH3CF3': {'C':2,'H':3,'F':3},
                 'CH3CHF2': {'C':2,'H':4,'F':2},
                 'CH3F':{'C':1,'H':3,'F':1},
                 'CH4': {'C':1,'H':4},
}

smiles = {'C2H5F':'CCF',
         'C2H6':'CC',
         'CH2F2': 'FCF',
         'CH2FCH2F':'FCCF',
         'CH2FCHF2': 'FCC(F)F',
         'CH3CF3': 'CC(F)(F)F',
         'CH3CHF2': 'CC(F)F',
         'CH3F': 'CF',
         'CH4': 'C',
          'O2': '[O][O]',
}

In [6]:
#get the species ID's of each of the species in the blends
#let's save this to a dictionary that we can reference in functions below

mapping = dict()


for blend in list_of_blends: 
    
    try:

        match = re.findall('([A-Z0-9]+)_([A-Z0-9]+)',blend)
        spec1 = match[0][0]
        spec2 = match[0][1]

        if len(match)>2:
            print(f'More than two species found: {blend}')

        direc = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts/{blend}/chemkin/'
        spcs, rxns = rmgpy.chemkin.load_chemkin_file(f'{direc}dups/dup_chem.inp',f'{direc}species_dictionary.txt', f'{direc}tran.dat' )
        
        for_this_blend=[]

        for species in spcs: 
            if species.smiles == smiles[spec1]: 
                for_this_blend.append(species.to_chemkin())
            if species.smiles == smiles[spec2]: 
                for_this_blend.append(species.to_chemkin())

        #did this in a separate list because I want to make sure O2 is last in the list
        for species in spcs: 
            if species.smiles == smiles['O2']: 
                for_this_blend.append(species.to_chemkin())

 
        #add to dictionary
        mapping[blend] = for_this_blend

    except: 
    
        print(blend)
        #print(e)

#print(mapping)


print(len(mapping.keys()), len(list_of_blends))


CH2FCH2F_CH3CHF2


CH3CF3_CH2FCHF2


19 21


In [22]:
### GIVING CHEMKIN ERRORS FOR TWO

blend_list = set(list_of_blends)
map_ = set(mapping.keys())
blend_list.difference(map_)

{'CH2FCH2F_CH3CHF2', 'CH3CF3_CH2FCHF2'}

In [11]:
#let's try to fix the errors

cant_do = ['CH2FCH2F_CH3CHF2']#, 'CH3CF3_CH2FCHF2']

mapping = dict()


for blend in cant_do: 

    match = re.findall('([A-Z0-9]+)_([A-Z0-9]+)',blend)
    spec1 = match[0][0]
    spec2 = match[0][1]

    if len(match)>2:
        print(f'More than two species found: {blend}')

    direc = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts/{blend}/chemkin/'
    spcs, rxns = rmgpy.chemkin.load_chemkin_file(f'{direc}dups/dup_chem-Copy1.inp',f'{direc}species_dictionary.txt', f'{direc}tran.dat' )

    for_this_blend=[]

    for species in spcs: 
        if species.smiles == smiles[spec1]: 
            for_this_blend.append(species.to_chemkin())
        if species.smiles == smiles[spec2]: 
            for_this_blend.append(species.to_chemkin())

    #did this in a separate list because I want to make sure O2 is last in the list
    for species in spcs: 
        if species.smiles == smiles['O2']: 
            for_this_blend.append(species.to_chemkin())


    #add to dictionary
    mapping[blend] = for_this_blend


#print(mapping)


print(len(mapping.keys()), len(list_of_blends))


ChemkinError: Could not read collider efficiencies for reaction: H2(8) <=> H(5) + H(5).
The following line was parsed incorrectly:
AR(10)/0.00/ HE(11)/0.00/ N2/1.01/ H2(8)/2.55/ H2O(9)/12.02/ CO(14)/1.95/ CO2(15)/3.83/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H6(32)/3.00/
(Case-preserved tokens: ['Ar(10)', '0.00', ' He(11)', '0.00', ' N2', '1.01', ' H2(8)', '2.55', ' H2O(9)', '12.02', ' CO(14)', '1.95', ' CO2(15)', '3.83', ' CH4(4)', '2.00', ' CH2O(21)', '2.50', ' CH3OH(27)', '3.00', ' C2H6(32)', '3.00', ''] )

In [25]:
new_list_of_blends = list(map_)

In [26]:
import os 
import re 
import cantera as ct



def make_gas(blend_name):
    print(blend_name)
    block = \
    f'''
directory = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts/{blend_name}/chemkin/dups/dup_chem.cti'
gas = ct.Solution(directory)
blend_name = '{blend_name}'

vol_frac_list = list(np.linspace(0.025, 0.25, 30))

results = dict()


#this part calculates the flame calcs for all of the vol_frac list, using the previous vol frac count as a guess for the current one
for i in  range(len(vol_frac_list)):
    try:

        string = f'****************************starting new volume fraction **************************'
        print(string)
        
        x = vol_frac_list[i]
        norm_ox = (1-x)*.21
'''
    return block

def make_vol_frac_dict(label_list,num):
    print(label_list)
    '''Makes a volume fraction dictionary, assuming there are equal parts fuel in the blend'''

    vol_frac_dict = \
    '''
        vol_frac_dict = {'''
    for label in label_list[:-1]:
        print(label)
        vol_frac_dict += f"'{label}': (x/{num}/norm_ox), "
    vol_frac_dict += f"'{label_list[-1]}':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox"
    vol_frac_dict += '}'
    
    block2 = \
    '''
        gas.TPX = To, Po, vol_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.max_time_step_count = 900
        loglevel = 1 
        try:
            if i!=0:
                d = f'data/{vol_frac_list[i-1]}.csv'
                if os.path.exists(d):  
                    arr2 = ct.SolutionArray(gas)
                    arr2.read_csv(d)
                    flame.set_initial_guess(data=arr2)
                    print(' initial guess has been set')
        except: 
            print('initial guess not set for this volume fraction')
        
        
        #"False" stops the calculation from retrying over and over
        flame.solve(loglevel=loglevel, auto=False)
        Su = flame.velocity[0]
        results[x] = Su
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        df1.to_csv(f'data/{x}.csv', index=False)            
            
    except Exception as e: 
        print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
        pass

vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)

    
with open('blend_flamespeeds.csv', 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(vol_fracs)
    writers.writerow(flame_speeds)
        
'''
    vol_frac_dict += block2
    return vol_frac_dict


def make_bash_script(label_list, time_limit, node):
    bash_script = f'''#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time={time_limit}-00:00:00
#SBATCH --job-name=fc_{'_'.join(label_list[0:-1])}
#SBATCH --error=error_fc.slurm.log
#SBATCH --output=output_fc.slurm.log
##SBATCH --cpus-per-task=5
##SBATCH --mem-per-cpu=8Gb
##SBATCH --ntasks=1 
##SBATCH --array=1
#SBATCH --partition={node}

source activate cantera_env
python flame_speed_calc.py

'''
    return bash_script

In [30]:
def make_blends(list_of_blend_files, num, dictionary_of_species):
    """Proved with a list of blend files, 
    specify the number of fuel species in the blend (num), this will run flame speed calc on all of them.
    
    example: make_blend([list using os.listdir(directory)])
    """
    

    for file in list_of_blend_files: 
             
            
            
        flame_speed_calc = '''

from os import uname
import cantera as ct
import numpy as np
import pandas as pd
import os
import csv

print("Running Cantera Version: " + str(ct.__version__))

To = 298
Po = 1e5 # ct.one_atm

    '''    
            
            
        
        print(f'Now working on: {file}')

        #now make the damn thing
        flame_speed_calc += make_gas(file)
        flame_speed_calc += make_vol_frac_dict(mapping[file],num)
        bash_script = make_bash_script(mapping[file], '7', 'west')
        directory_to_put_script =f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts/{file}'
        with open(os.path.join(directory_to_put_script,f'flame_calc.py'),'w') as f: 
            for l in flame_speed_calc: 
                f.write(l)
        with open(os.path.join(directory_to_put_script,f'flame_speed_run.sh'),'w') as f: 
            for l in bash_script: 
                f.write(l)
                
        os.makedirs(os.path.join(directory_to_put_script,'data'),exist_ok=True)
  



In [31]:
make_blends(new_list_of_blends, 2, for_this_blend)

Now working on: CH3CHF2_CH2FCHF2
CH3CHF2_CH2FCHF2
['CH3CHF2(1)', 'C2H3F3(2)', 'O2(3)']
CH3CHF2(1)
C2H3F3(2)
Now working on: CH2FCH2F_CH3F
CH2FCH2F_CH3F
['C2H4F2(1)', 'CH3F(2)', 'O2(3)']
C2H4F2(1)
CH3F(2)
Now working on: C2H5F_CH3CHF2
C2H5F_CH3CHF2
['C2H5F(1)', 'CH3CHF2(2)', 'O2(3)']
C2H5F(1)
CH3CHF2(2)
Now working on: C2H5F_CH3F
C2H5F_CH3F
['C2H5F(1)', 'CH3F(2)', 'O2(3)']
C2H5F(1)
CH3F(2)
Now working on: CH3CF3_C2H5F
CH3CF3_C2H5F
['CH3CF3(1)', 'C2H5F(2)', 'O2(3)']
CH3CF3(1)
C2H5F(2)
Now working on: CH3CF3_CH2FCH2F
CH3CF3_CH2FCH2F
['CH3CF3(1)', 'C2H4F2(2)', 'O2(3)']
CH3CF3(1)
C2H4F2(2)
Now working on: C2H5F_CH2FCH2F
C2H5F_CH2FCH2F
['C2H5F(1)', 'C2H4F2(2)', 'O2(3)']
C2H5F(1)
C2H4F2(2)
Now working on: CH2FCH2F_CH2FCHF2
CH2FCH2F_CH2FCHF2
['C2H4F2(1)', 'C2H3F3(2)', 'O2(3)']
C2H4F2(1)
C2H3F3(2)
Now working on: CH3CF3_CH3CHF2
CH3CF3_CH3CHF2
['CH3CF3(1)', 'CH3CHF2(2)', 'O2(3)']
CH3CF3(1)
CH3CHF2(2)
Now working on: CH2FCH2F_CH2F2
CH2FCH2F_CH2F2
['C2H4F2(1)', 'CH2F2(2)', 'O2(3)']
C2H4F2(1)
CH2F2

In [32]:
' '.join(new_list_of_blends)
    

'CH3CHF2_CH2FCHF2 CH2FCH2F_CH3F C2H5F_CH3CHF2 C2H5F_CH3F CH3CF3_C2H5F CH3CF3_CH2FCH2F C2H5F_CH2FCH2F CH2FCH2F_CH2FCHF2 CH3CF3_CH3CHF2 CH2FCH2F_CH2F2 C2H5F_CH2F2 CH2F2_CH3CF3 C2H5F_CH2FCHF2 CH3F_CH3CHF2 CH2F2_CH3F CH3F_CH3CF3 CH2FCHF2_CH3F CH3CHF2_CH2F2 CH2FCHF2_CH2F2'

In [36]:
#scratchpad 

import pandas as pd


In [37]:
df = pd.read_csv('/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/concatenate_equal_parts/flamespeeds/C2H5F_CH2F2/concatenated_blend.csv')

In [52]:
[float(i) for i in df.columns]

[0.04051724137931034,
 0.04827586206896552,
 0.05603448275862069,
 0.06379310344827586,
 0.07155172413793104,
 0.0793103448275862,
 0.08706896551724139,
 0.09482758620689655,
 0.10258620689655173,
 0.1103448275862069,
 0.11810344827586206,
 0.12586206896551724,
 0.1336206896551724,
 0.1413793103448276,
 0.14913793103448275,
 0.1568965517241379,
 0.1646551724137931,
 0.17241379310344826,
 0.18017241379310345,
 0.1879310344827586,
 0.1956896551724138,
 0.20344827586206896,
 0.21120689655172412,
 0.2189655172413793,
 0.22672413793103446,
 0.23448275862068965,
 0.2422413793103448,
 0.25]

In [48]:
df.iloc[0, header=False]

SyntaxError: invalid syntax (1663844760.py, line 1)

In [61]:
df.values[0,:]

array([0.00658779, 0.02129391, 0.05058648, 0.08888529, 0.13107889,
       0.17320337, 0.21051773, 0.23377054, 0.24024206, 0.22522235,
       0.18797997, 0.13846322, 0.09104891, 0.06215019, 0.04679535,
       0.03627221, 0.02869491, 0.02305072, 0.01887064, 0.01576279,
       0.01350104, 0.01179832, 0.01049464, 0.00948148, 0.00868279,
       0.0080087 , 0.00743279, 0.00693475])

In [62]:
df.values * 100

array([[ 0.65877918,  2.12939054,  5.0586475 ,  8.8885285 , 13.10788897,
        17.32033687, 21.05177308, 23.37705404, 24.02420647, 22.52223473,
        18.7979966 , 13.84632226,  9.10489059,  6.21501935,  4.67953524,
         3.62722059,  2.86949107,  2.30507207,  1.88706418,  1.57627895,
         1.35010386,  1.1798322 ,  1.04946411,  0.94814777,  0.86827917,
         0.80087033,  0.74327933,  0.69347526]])

In [54]:
try

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().